#정형 vs 비정형
---
구조화 된 데이터들은 데이터베이스나 엑셀에 표현하기 용이하다. 이러한 데이터를 정형 데이터(structured data)라 한다. 반대로 텍스트 데이터나 사진처럼 엑셀로 표현하기 어려운 데이터를 비정형 데이터(unstructured date)라 한다. 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘이 앙상블 학습이다. 이 알고리즘은 대부분 결정 트리를 기반으로 만들어져 있다. 반대로 비정형 데이터에는 신경망 알고리즘을 사용한다.

#랜덤 포레스트(Random Forest)
---
랜덤 포레스트는 앙상블 학습의 대표 주자 중 하나로안정적인 성능 덕분에 널리 사용되고 있다. 랜덤 포레스트는 결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만든 후 각 결정 트리의 예측을 사용해 최종 예측을 만든다. 랜덤 포르스트는 각 트리를 훈련하기 위해 랜덤한 데이터를 만드는데, 훈련 데이터에서 중복을 허용하여 샘플을 추출한다. 이렇게 만들어진 샘플을 **부트스트랩 샘플(bootstrap sample)**이라고 한다. 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같다. 각 노드를 분할할 때, RandomForestClassifier 분류 모델은 일반적으로 전체 특성 개수의 제곱근만큼의 특성을 무작위로 선택하여 이 중에서 최선의 분할을 찾는다. 반면 RandomForestRegressor 회귀 모델은 전체 특성을 사용한다.  

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

wine = pd.read_csv('https://bit.ly/wine_csv_data') # 데이터 불러온 후 세트 나누기
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

rf = RandomForestClassifier(n_jobs = -1, random_state = 42) # 교차 검증 수행
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 과대적합

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_) # 특성의 일부를 랜덤하게 선택하여 훈련하기 때문에 하나의 특성에 과도하게 집중하지 않는다.

[0.23167441 0.50039841 0.26792718]


랜덤 포레스트는 훈련 세트에서 중복을 허용하여 샘플을 만들기 때문에 남는 샘플이 존재한다. 이런 샘플을 **OOB(out of bag)** 샘플이라고 한다. 이 남는 샘플이 검증 세트의 역할을 한다.

In [ ]:
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


#엑스트라 트리(Extra Tree)
---
엑스트라 트리는 랜덤 포레스트와 매우 비슷하게 동작한다. 차이점은, 엑스트라 트리는 부트스트랩 샘플을 사용하지 않는다는 것이다. 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다. 하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만, 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있다.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


#그래이디언트 부스팅(Gradient Boosting)
---
그래이디언트 부스킹은 깊이가 얕은 결정 트리를 사용하여 이진 트리의 오차를 보완하는 방식으로 앙상블 하는 방법이다. 경사 하강법을 사용하여 트리를 앙상블에 추가한다. 분류에서는 로지스틱 손실 함수를, 회귀에서는 평균 제곱 오차 함수를 사용한다. 경사 하강법에서 손실 함수의 낮은 곳으로 조금씩 이동하듯이, 그래이디언트 부스팅도 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동하기 때문에 깊이가 얕은 트리를 사용한다. 사이킷런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리 100개를 사용한다. 깊이가 얕기 떄문에 과대적합에 강하고 높은 일반화 성능을 기대할 수 있다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 거의 과대적합 되지 않는다.

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42) # 학습률을 증가시키고 트리의 개수를 늘려 성능 향상
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


#히스토그램 기반 GB(Histogram-based GBoosting)
---
히스토그램 기반 그래이디언트 부스팅은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘이다. 이 알고리즘은 입력 특성을 256개 구간으로 나누기 때문에 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
train_result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
test_result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print('훈련 세트의 특성 중요도:', train_result.importances_mean, '테스트 세트의 특성 중요도:', test_result.importances_mean)
hgb.score(test_input, test_target)

훈련 세트의 특성 중요도: [0.08876275 0.23438522 0.08027708] 테스트 세트의 특성 중요도: [0.05969231 0.20238462 0.049     ]


0.8723076923076923

In [ ]:
from xgboost import XGBClassifier # 사이킷런 말고도 히스토_그래디언트를 구현한 XGBoost 라이브러리

xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [ ]:
from lightgbm import LGBMClassifier # 사이킷런 말고도 히스토_그래디언트를 구현한 LightGBM 라이브러리

lgb = LGBMClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
